In [33]:
import pandas as pd 

file_path = 'C:/Users/ASUS/AI_Journey/feature_selection/data.csv/data.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
df.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [69]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


# Encode the target column 'action_type'
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['shot_made_flag'])

columns_to_keep = ['combined_shot_type', 'season', 'shot_type', 'shot_zone_area',\
    'shot_zone_basic', 'shot_zone_range','action_type','team_name','game_date','matchup','opponent']
# Select the feature and dropping irrelevant columns
X = df[columns_to_keep].dropna()


# Drop rows with any NaN values
X_cleaned = X.dropna()
# Drop columns that contain any NaN values
X_cleaned = X.dropna(axis=1)

# One-Hot Encoding for categorical features
categorical_columns = ['combined_shot_type', 'season', 'shot_type', 'shot_zone_area',\
    'shot_zone_basic', 'shot_zone_range','action_type','team_name','game_date','matchup','opponent']
X_encoded = pd.get_dummies(X_cleaned, columns=categorical_columns,drop_first=True) #One hot encoded the categorical feature

#  Split the data into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X_encoded,y,test_size=0.2,random_state=42)

In [71]:
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Select the top 10 feature based on the F-Statistic
selector = SelectKBest(score_func=f_classif,k=10)
X_train_selected = selector.fit_transform(X_train,y_train)

# Get the  mask of selected features
selected_features = X_train.columns[selector.get_support()]

print(f"Select Features: \n",selected_features)


# Transform the test set to keep the selected features
X_test_selected = selector.transform(X_test)

# Train a RandomForest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train_selected, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy with Selected Features: {accuracy:.2f}")

c:\Users\ASUS\AI_Journey\my_working_env\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  84   91  137  148 1637] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\ASUS\AI_Journey\my_working_env\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Select Features: 
 Index(['combined_shot_type_Dunk', 'combined_shot_type_Jump Shot',
       'shot_type_3PT Field Goal', 'shot_zone_area_Center(C)',
       'shot_zone_basic_Restricted Area', 'shot_zone_range_24+ ft.',
       'shot_zone_range_Less Than 8 ft.', 'action_type_Driving Layup Shot',
       'action_type_Jump Shot', 'action_type_Slam Dunk Shot'],
      dtype='object')
Model Accuracy with Selected Features: 0.56
